<a href="https://colab.research.google.com/github/ravin235/hamoye/blob/main/StageB_Quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Importing all the required library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
import numpy as np

In [4]:
#Loading the datasets from github raw content
df = pd.read_csv('https://raw.githubusercontent.com/ravin235/hamoye-datasets/main/energydata_complete%20(1).csv')

In [5]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


Q12: From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) 
and the temperature outside the building (y = T6). What is the R^2 value in two d.p.?

In [6]:
#assigning T2 as feature variable and T6 as target variable
tempX_df = df[['T2']]
tempY_df = df[['T6']]

In [8]:
from sklearn.model_selection import train_test_split
tempx_train,tempx_test,tempy_train,tempy_test = train_test_split(tempX_df,tempY_df,test_size=0.3,random_state=1)

In [10]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(tempx_train, tempy_train)
temp_predicted_values = linear_model.predict(tempx_test)

In [11]:
from sklearn.metrics import r2_score
temp_r2_score = r2_score(tempy_test, temp_predicted_values)
round(temp_r2_score,3)

0.646

Q13: Normalize the dataset using the MinMaxScaler after removing the following columns: [“date”, “lights”]. The target variable is “Appliances”. Use a 70-30 train-test set split with a random state of 42 (for reproducibility). Run a multiple linear regression using the training set and evaluate your model on the test set. Answer the following questions:

What is the Mean Absolute Error (in two decimal places)?

In [12]:
df.drop(columns=['date','lights'],inplace=True)

In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)
features_df = normalised_df.drop(columns = ['Appliances'])
target_df = normalised_df['Appliances']

In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(features_df,target_df,test_size=0.3,random_state=42)

In [15]:
#Let's select a model for training
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(x_train, y_train)
predicted_values = linear_model.predict(x_test)

In [16]:
#Measure the accuracy of the model
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test,predicted_values)
round(mae,3)

0.05

Q14: What is the Residual Sum of Squares (in two decimal places)?

In [17]:
rss = np.sum(np.square(y_test-predicted_values))
round(rss,2)

45.35

Q15: What is the Root Mean Squared Error (in three decimal places)?

In [18]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test,predicted_values))
round(rmse,3)

0.088

Q16: What is the Coefficient of Determination (in two decimal places)?

In [19]:
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score,2)

0.15

Q17: Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?

In [20]:
def get_weights_df(model, feat, col_name):
  weights =  pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features',col_name]
  weights_df[col_name].round(3)
  return weights_df

In [21]:
linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight')

In [32]:
linear_model_weights.values[0][0],linear_model_weights.values[-1][0]

('RH_2', 'RH_1')

Q18: Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

In [33]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)

Ridge(alpha=0.4)

In [34]:
ridge_predicted_values = ridge_reg.predict(x_test)

In [37]:
ridge_rmse = np.sqrt(mean_squared_error(y_test,ridge_predicted_values))
round(ridge_rmse,3)

0.088

Q20: What is the new RMSE with the lasso regression? (Answer should be in three (3) decimal places)

In [40]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

Lasso(alpha=0.001)

In [41]:
lasso_predicted_values = lasso_reg.predict(x_test)

In [42]:
lasso_rmse = np.sqrt(mean_squared_error(y_test,lasso_predicted_values))
round(lasso_rmse,3)

0.094

Q19: Train a lasso regression model with an alpha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?

In [49]:
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')
len(lasso_weights_df[lasso_weights_df['Lasso_weight']!=0])

4